# Multipath TCP experiments

In this notebook we create a multipath topology in FABRIC with the intent of evaluating various multipath protocols such us MPTCP with different link types and application traffic.

## Multipath Topology

![multipath-diagram.svg](multipath-diagram.svg)

## Set up environment

It is assumed that the keys, the FABRIC config file, and the SSH config file are already created (in the "Configure your environment" notebook) using those default settings (paths, etc).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

In [ ]:
import os
slice_name="mptcp-base" + os.getenv('NB_USER')

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

## Use previously reserved resources (if needed)

See if you already have the slice

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

## Create slice and reserve resources (only if you don't already have)

Next, we’ll select a random FABRIC site for our experiment. We’ll make sure to get one that has sufficient capacity for the experiment we’re going to run.

Once we find a suitable site, we’ll print details about available resources at this site.


In [ ]:
exp_requires = {'core': 4*6, 'nic':12, 'ram':32*6}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
         (fablib.resources.get_ram_available(site_name) > 1.2*exp_requires['ram']) and
         (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break

fablib.show_site(site_name)

Add nodes to the slice (note: use Ubuntu 18 because the MPTCP kernel is based on kernel 4.19):

In [ ]:
# to be deleted, since next cell sets up the nodes !!!
n_c  = slice.add_node(name="client",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_s  = slice.add_node(name="server",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r1 = slice.add_node(name="router1",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e1 = slice.add_node(name="emulator1", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r2 = slice.add_node(name="router2",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e2 = slice.add_node(name="emulator2", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')

In [ ]:
# this cell sets up the nodes
node_names = ["client", "server", "router1", "emulator1", "router2", "emulator2"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')

Add network interfaces to the nodes:

In [ ]:
# to be deleted after fixing the next cell !!!
if_c1     =  n_c.add_component(model="NIC_Basic", name="if_c1").get_interfaces()[0]
if_c2     =  n_c.add_component(model="NIC_Basic", name="if_c2").get_interfaces()[0]
if_s1     =  n_s.add_component(model="NIC_Basic", name="if_s1").get_interfaces()[0]
if_s2     =  n_s.add_component(model="NIC_Basic", name="if_s2").get_interfaces()[0]
if_e1_l   = n_e1.add_component(model="NIC_Basic", name="if_e1_l").get_interfaces()[0]
if_e1_r   = n_e1.add_component(model="NIC_Basic", name="if_e1_r").get_interfaces()[0]
if_r1_l   = n_r1.add_component(model="NIC_Basic", name="if_r1_l").get_interfaces()[0]
if_r1_r   = n_r1.add_component(model="NIC_Basic", name="if_r1_r").get_interfaces()[0]
if_e2_l   = n_e2.add_component(model="NIC_Basic", name="if_e2_l").get_interfaces()[0]
if_e2_r   = n_e2.add_component(model="NIC_Basic", name="if_e2_r").get_interfaces()[0]
if_r2_l   = n_r2.add_component(model="NIC_Basic", name="if_r2_l").get_interfaces()[0]
if_r2_r   = n_r2.add_component(model="NIC_Basic", name="if_r2_r").get_interfaces()[0]

In [ ]:
# to be fixed: this cell sets up the network links !!! see the hello, fabric instructions https://github.com/teaching-on-testbeds/hello-fabric/blob/main/hello_fabric.ipynb 

nets = [
    {"name": "net1a",   "nodes": ["romeo", "router"]},
    {"name": "net1b",  "nodes": ["router", "juliet"]}
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

and show them:

add Ethernet networks to the slice:

In [ ]:
net_1_c_e = slice.add_l2network(name='net_1_c_e', type='L2Bridge', interfaces=[if_c1,    if_e1_l])
net_2_c_e = slice.add_l2network(name='net_2_c_e', type='L2Bridge', interfaces=[if_c2,    if_e2_l])
net_1_e_r = slice.add_l2network(name='net_1_e_r', type='L2Bridge', interfaces=[if_e1_r,  if_r1_l])
net_2_e_r = slice.add_l2network(name='net_2_e_r', type='L2Bridge', interfaces=[if_e2_r,  if_r2_l])
net_1_r_s = slice.add_l2network(name='net_1_r_s', type='L2Bridge', interfaces=[if_r1_r,  if_s1])
net_2_r_s = slice.add_l2network(name='net_2_r_s', type='L2Bridge', interfaces=[if_r2_r,  if_s2])

and see them:

In [ ]:
print(f"{slice.list_interfaces()}")

Note: Basic NICs may claim 0 bandwidth but are 100 Gbps shared by all Basic
NICs on the host.

Submit slice (reserve resources):

Now we are ready to reserve the resources and get the login details.

This step will take a little while, as we communicate with FABRIC to
reserve our requested resources.

-   As it runs, you will see the “State” of each node go from “Ticketed”
    to “Active” (and a “Management IP”, which may be IPv4 or IPv6, will
    be assigned to each one)
-   Then, you’ll wait a few more minutes for some `wait_ssh` and
    `post_boot_config` processes to run.
-   Next, it will show the MAC address and interface name of each of the
    “dataplane” interfaces you set up, as these come up

In [ ]:
slice.submit()

Our final slice status should be “StableOK”:

In [ ]:
print(f"{slice}")

we can block until all hosts are ready to login:

In [ ]:
slice.wait_ssh(progress=True)

and we can get login details for every node:

In [ ]:
for node in slice.get_nodes():
    print(f"{node}")

Note: you can open a terminal in the Jupyter environment and use these SSH commands there to log in to the nodes. But, add 

```
 -F ~/work/fabric_config/ssh_config
```

to each SSH command.

Test that resources are available over SSH - should return True:

In [ ]:
slice.test_ssh()

Note: now that the slice is reserved, the variables from the previous section (node and interface variables) are no longer useful. 

### Install MPTCP kernel on client and server

First, we download the packages to the Jupyter workspace (the nodes cannot retrieve them directly from Github because of [the IPv6 issue](https://learn.fabric-testbed.net/knowledge-base/using-ipv4-only-resources-like-github-or-docker-hub-from-ipv6-fabric-sites/).) 

In [ ]:
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-libc-dev_20220311125841-1_amd64.deb -O /tmp/linux-libc-dev_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-mptcp_v0.95.2_20220311125841-1_all.deb -O /tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb

Then we transfer over to the hosts and install.

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_file('/tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb', '/home/ubuntu/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb',   '/home/ubuntu/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-libc-dev_20220311125841-1_amd64.deb',               '/home/ubuntu/linux-libc-dev_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb',            '/home/ubuntu/linux-mptcp_v0.95.2_20220311125841-1_all.deb')
    slice.get_node(name=node_name).execute("sudo dpkg -i linux*.deb")
    slice.get_node(name=node_name).execute("sudo apt-get install -f")

Restart the nodes:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo reboot")

and wait for them to come up again:

In [ ]:
slice.wait_ssh(progress=True)

Check that MPTCP kernel is loaded (`4.19.X.mptcp`):

In [ ]:
for node_name in ["client", "server"]:
    output = slice.get_node(name=node_name).execute("uname -r")[0].strip()
    print(f"Kernel on {node_name}: {output}")
    
    if "mptcp" not in output: # halt notebook execution if kernel is not installed!
        raise Exception ("No MPTCP kernel on %s" % node_name)

### Configure network 

Now we can configure IP addresses on every network interface:

In [ ]:
print(f"{slice.list_interfaces()}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

slice.get_node(name='client').ip_addr_add(addr="192.168.10.2", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="client-if_c1-p1"))
slice.get_node(name='client').ip_addr_add(addr="192.168.20.2", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="client-if_c2-p1"))
                                          
slice.get_node(name='server').ip_addr_add(addr="192.168.3.1", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="server-if_s1-p1"))
slice.get_node(name='server').ip_addr_add(addr="192.168.4.1", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="server-if_s2-p1"))

slice.get_node(name='router1').ip_addr_add(addr="192.168.1.2", subnet=IPv4Network("192.168.1.0/24"), interface=slice.get_interface(name="router1-if_r1_l-p1"))
slice.get_node(name='router1').ip_addr_add(addr="192.168.3.2", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="router1-if_r1_r-p1"))
                                          
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.10.1", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="emulator1-if_e1_l-p1"))
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.1.1",  subnet=IPv4Network("192.168.1.0/24"),  interface=slice.get_interface(name="emulator1-if_e1_r-p1"))

slice.get_node(name='router2').ip_addr_add(addr="192.168.2.2", subnet=IPv4Network("192.168.2.0/24"), interface=slice.get_interface(name="router2-if_r2_l-p1"))
slice.get_node(name='router2').ip_addr_add(addr="192.168.4.2", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="router2-if_r2_r-p1"))
                                           
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.20.1", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="emulator2-if_e2_l-p1"))
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.2.1",  subnet=IPv4Network("192.168.2.0/24"),  interface=slice.get_interface(name="emulator2-if_e2_r-p1"))

## Set up multipath networking

### Client and server

Download and install `iproute-mptcp` on client and server:


In [ ]:
!git clone https://github.com/multipath-tcp/iproute-mptcp.git

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_directory('iproute-mptcp', '/home/ubuntu/')
    slice.get_node(name=node_name).execute("cd /home/ubuntu/iproute-mptcp; sudo make; sudo make install")

Install some other utility software on client and server:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo apt update; sudo apt -y install iperf3 net-tools moreutils")

Enable multipath kernel modules on client and server:

In [ ]:
cmds = """
# run sysctl commands to enable mptcp
sudo sysctl -w net.mptcp.mptcp_enabled=1 
sudo sysctl -w net.mptcp.mptcp_checksum=0

# load and configure mptcp congestion control algorithms
sudo modprobe mptcp_balia 
sudo modprobe mptcp_coupled 
sudo modprobe mptcp_olia 
sudo modprobe mptcp_wvegas

# configure mptcp congestion control algorithm to one
sudo sysctl -w net.ipv4.tcp_congestion_control=balia
"""
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute(cmds)

Bring up interfaces and add routes on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.10.1")
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.20.1")

Enable MPTCP in the experiment interfaces on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    slice.get_node(name="client").execute("sudo ip link set dev " + iface.get_os_interface() + " multipath on")

and disable on the control interface:

(note: currently broken, see https://learn.fabric-testbed.net/forums/topic/error-on-get_management_os_interface/
but it doesn't seem to matter)

In [ ]:
control_iface = slice.get_node(name="client").get_management_os_interface()
slice.get_node(name="client").execute("sudo ip link set dev " + control_iface + " multipath off")

Now, add MP routes on client:

In [ ]:
cmds = """
iface1=$(ifconfig | grep -B1 'inet 192.168.10.2' | head -n1 | cut -f1 -d:)
iface2=$(ifconfig | grep -B1 'inet 192.168.20.2' | head -n1 | cut -f1 -d:)

sudo ip rule add from 192.168.10.2 table 1 
sudo ip rule add from 192.168.20.2 table 2 
sudo ip route add 192.168.10.0/24 dev $iface1 scope link table 1
sudo ip route add 192.168.20.0/24 dev $iface2 scope link table 2 
sudo ip route add 192.168.3.0/24 via 192.168.10.1 dev $iface1 table 1 
sudo ip route add 192.168.4.0/24 via 192.168.20.1 dev $iface2 table 2
"""
slice.get_node(name="client").execute(cmds)

Check configuration on client:

In [ ]:
client_ip_ad = slice.get_node(name="client").execute('ip addr')
client_ip_rt = slice.get_node(name="client").execute('ip route')
client_ip_r1 = slice.get_node(name="client").execute('ip route show table 1')
client_ip_r2 = slice.get_node(name="client").execute('ip route show table 2')
for output in [client_ip_ad, client_ip_rt, client_ip_r1, client_ip_r2]:
    print(*output, sep='\n')

Bring up interfaces and add routes on server:

In [ ]:
for iface in slice.get_node(name="server").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.3.2")
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.4.2")

Check configuration on server:

In [ ]:
server_ip_ad = slice.get_node(name="server").execute('ip addr')
server_ip_rt = slice.get_node(name="server").execute('ip route')
print(*server_ip_ad, sep='\n')
print(*server_ip_rt, sep='\n')

### Gateways

Bring up links:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    for iface in slice.get_node(name=node_name).get_interfaces():
        iface.ip_link_up()

Enable IP forwarding:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    slice.get_node(name=node_name).execute("sudo sysctl -w net.ipv4.ip_forward=1")

Add some routes throughout the network:

In [ ]:
slice.get_node(name="emulator1").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.1.2")
slice.get_node(name="emulator2").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.2.2")
slice.get_node(name="router1").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.1.1")
slice.get_node(name="router2").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.2.1")

Add delay at emulator nodes on the "right" side of the network:

In [ ]:
em_iface_r = slice.get_node(name="emulator1").get_interface(name="emulator1-if_e1_r-p1").get_os_interface()
slice.get_node(name="emulator1").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')
em_iface_r = slice.get_node(name="emulator2").get_interface(name="emulator2-if_e2_r-p1").get_os_interface()
slice.get_node(name="emulator2").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')

Add capacity limit on the router nodes on the "right" side of the network"

In [ ]:
cmds = """
sudo tc qdisc del dev IFACE root # don’t worry if you get RTNETLINK  error
sudo tc qdisc replace dev IFACE root handle 1: htb default 3 
sudo tc class add dev IFACE parent 1: classid 1:3 htb rate 100mbit 
sudo tc qdisc add dev IFACE parent 1:3 handle 3: bfifo limit 375000 
"""

rt_iface_r = slice.get_node(name="router1").get_interface(name="router1-if_r1_r-p1").get_os_interface()
slice.get_node(name="router1").execute(cmds.replace("IFACE", rt_iface_r))
rt_iface_r = slice.get_node(name="router2").get_interface(name="router2-if_r2_r-p1").get_os_interface()
slice.get_node(name="router2").execute(cmds.replace("IFACE", rt_iface_r))

View all the routes:

In [ ]:
for node_name in ["client", "server", "emulator1", "emulator2", "router1", "router2"]: 
    ip_rt = slice.get_node(name=node_name).execute('ip route')
    print("Routes at " + node_name + ":")
    print(*ip_rt, sep='\n')

### Test multipath configuration

Start an iperf3 server on the server node (in daemon mode -D option, so this command won't block).
Then start an iperf3 client on the client node.

In [ ]:
slice.get_node(name="server").execute("iperf3 -s -1 -D")

In [ ]:
iperf_out = slice.get_node(name="client").execute("iperf3 -f m -c 192.168.3.1 -C 'balia' -P 1 -t 20")
print(*iperf_out, sep='\n')

Confirm that you get about 200 Mbps throughput.

## Renew your slice lease if needed

In [ ]:
import datetime
from datetime import timedelta

now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=10)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

See the lease end date which may not print most up to date value here. You can also confirm the lease end date from fabrib web interface.

In [ ]:
print(f"{slice}")

## Run Experiment: Bulk Transfer

### Set up experiment

Select Congestion Control algorithm in client and server

In [ ]:
#ccAlgos=["balia", "coupled", "olia", "wvegas"] # we can experiment with other CC algos later
ccAlgo="balia"
slice.get_node(name="client").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="server").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="client").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)
slice.get_node(name="server").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)

Copy Scripts and Trace files to router1 and router2

In [ ]:
MPTracesRootFolder=os.environ['HOME'] + "/work/MP-Traces"
nodeHomeFolder="/home/ubuntu"

In [ ]:
for node_name in ["router1", "router2"]:
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/Scripts", nodeHomeFolder)
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/Traces", nodeHomeFolder)    

### Execute experiment

#### Bulk transfer (using mptcp trace pair, memoryless random process, and static process)

In [ ]:
# get the current time, to use when naming the directory to hold results/outputs of the experiment
import datetime
from datetime import timedelta

 
print("starting a new Bulk Transfer experiment set...")

now = datetime.datetime.now(datetime.timezone.utc)
outputDir="RawResults_" + now.strftime("%Y_%m_%d_%H_%M_%S")
print("creating outputDir " + outputDir)
slice.get_node(name="client").execute("mkdir -p " + outputDir) 

# select path and trial
# positive correlation between cellular and wifi paths: Pearson R  of 0.19499423234541421
path="7" 
trial="1"
meanRateWifi=24.36 # Mbps, mean and stddev of path 7 real measurements as read from multipath.ipyb script
stdDevWifi=17.95
meanRateCellular=48.02 # Mbps
stdDevCellular=16.19
# negative correlation between cellular and wifi paths: Pearson R of -0.24126157391622494
# path= "8"
# trial="3"
# meanRateWifi=25.44 # Mbps,  mean and stddev of path 8 real measurements as read from multipath.ipyb script
# stdDevWifi=22.57
# meanRateCellular=43.07 # Mbps
# stdDevCellular=14.03

# get the trace pair file names to feed into the routers
wifiTraceFile=nodeHomeFolder + "/Traces/" + path + "_" + trial + "_wifi.csv"  
cellularTraceFile=nodeHomeFolder + "/Traces/"+ path + "_" + trial + "_cellular.csv" 
print("\twith trace files: " + cellularTraceFile + "  " + wifiTraceFile + "  ")


In [ ]:
for expType in ["ST"]: #TP: using trace pairs at the routers, MR: memoryless random rates at the routers, ST: static (use stdDev as 0)
    print("running experiment Type: " + expType)
    for expNo in ["1","2","3","4","5"]: 
        print("\t____running experiment No " + expNo + "____" )
        outputFileName=outputDir + "/" + "type"+ expType + "_CC" + ccAlgo + "_exp" + expNo + "_path" + path + "_trial" + trial
        print("\toutput file: " + outputFileName)
        if (expType == "TP"):
            # start scripts in router1 and router2 (in non-blocking way using *at* command) 
            # so the bandwidths at the routers are regulated based on the collected
            # trace pairs (wifi trace on router 1 and cellular trace in router2, respectively)
            # important note: make sure to run the script with /bin/bash to make it work with at command
            router1Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVary.sh " + wifiTraceFile + " | at now"
            router2Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVary.sh "+ cellularTraceFile +" | at now"
        elif (expType == "MR"):
            # start scripts in router1 and router2 (in non-blocking way using *at* command)                         
            # important note: make sure to run the script with /bin/bash to make it work with at command
            router1Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVaryRandom.sh " + str(meanRateWifi) + " " + str(stdDevWifi) + "| at now"
            router2Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVaryRandom.sh "+ str(meanRateCellular) + " " + str(stdDevCellular) + "| at now"
        elif (expType == "ST"): 
            # start scripts in router1 and router2 (in non-blocking way using *at* command)                         
            # important note: make sure to run the script with /bin/bash to make it work with at command
            router1Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVaryRandom.sh " + str(meanRateWifi) + " " + " 0 | at now"
            router2Cmd="echo /bin/bash " + nodeHomeFolder + "/Scripts/tputVaryRandom.sh "+ str(meanRateCellular) + " " +  " 0 | at now"
        else:   
            print("\tError: Wrong expType!")
            break
        print("\texecuting router commands: " + router1Cmd + " " + router2Cmd)
        slice.get_node(name="router1").execute(router1Cmd)
        slice.get_node(name="router2").execute(router2Cmd)

        print("\trunning the iperf at the server...")
        slice.get_node(name="server").execute('iperf3 -s -i 0.1 -D -1') # run as daemon and stop after 1 client connection

        clientCmd="iperf3 -f m -c 192.168.3.1 -C '" + ccAlgo + "' -P 1 -i 1 -t 95" + " > " + outputFileName 
        print("\tclient iperf command: " + clientCmd)
        print("\trunning iperf at the client...") 
        slice.get_node(name="client").execute(clientCmd)
        #iperf_out = slice.get_node(name="client").execute(clientCmd)
        #print(*iperf_out, sep='\n')

        print("\tkilling the trace pair feeding scripts at both the routers to end this experiment...")        
        killCmd="kill -9 $(ps -ef | grep -i tput | awk '{print $2}')"
        slice.get_node(name="router1").execute(killCmd)
        slice.get_node(name="router2").execute(killCmd)
        
print("finished running Bulk Transfer experiments above.")       

### Retrieve data from experiment

In [ ]:
slice.get_node(name="client").download_directory('/home/fabric/work/' ,  outputDir )

## Data analysis experiment: Bulk Transfer

In [ ]:
# to do 11/17/2022: create a table
# path, trial, TPmeanData, TPstdErr, MRmeanData, MRstdErr,  (aggregated over multiple experiments)

import re
from os import listdir
from os.path import isfile, join
#outputDir="RawResults_2022_11_03_13_12_27"
outputDir="RawResults_2022_11_17_12_52_24"

localOutputDir='/home/fabric/work/' + outputDir
print("Started Data Analysis for files in " + localOutputDir + "...")
rawOutputFiles = [f for f in listdir(localOutputDir) if isfile(join(localOutputDir, f))]
print(rawOutputFiles)

resultsArray=[]
for rawOutputFile in rawOutputFiles:     
    if (rawOutputFile.startswith('type')  ):
        print("\tprocessing raw output file " + rawOutputFile)
        filenameArray=re.split('_',rawOutputFile)
        etype=re.split('type', filenameArray[0])[1]
        CC=re.split('CC', filenameArray[1])[1]
        expNo=int(re.split('exp', filenameArray[2])[1])
        path=re.split('path', filenameArray[3])[1]
        trial=int(re.split('trial', filenameArray[4])[1])
        timeArray=[]
        tputArray=[]        
        with open(localOutputDir + "/" + rawOutputFile, 'r') as f:           
            for line in f.readlines():
                    if 'Mbits/sec' in line and 'sender' not in line and 'receiver' not in line:
                        timeArray.append(float(re.split('\s+',re.split('-', line)[1])[0]))                    
                        tputArray.append(float(re.split('\s+',line)[6]))
                    if 'Mbits/sec' in line and 'receiver' in line: 
                        avgTput=float(re.split('\s+', line)[6])                     
            resultsArray.append({"etype":etype,
                                 "CC":CC,
                                 "expNo":expNo,
                                 "path":path,
                                 "trial":trial,
                                 "avgTput":avgTput,
                                 "time":timeArray, 
                                 "tput": tputArray})
           # print(resultsArray)

In [ ]:
print(resultsArray)
import pandas as pd
df= pd.DataFrame(resultsArray)
df

In [ ]:
#g=df.groupby(['path','trial','etype'])['avgTput']
g=df.groupby(['path','trial','etype']).agg({'avgTput':['mean','sem']}) # sem: standard error of mean
g.reset_index(inplace=True, drop=False )
g

## Delete slice

In [ ]:
#fablib.delete_slice(slice_name)